In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-07 10:01:16.238314


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [3]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Artists

In [39]:
%load_ext autoreload
%autoreload

def parseArtists(arts, force=False):
    for modVal in range(18,19):
        arts.parseArtistModValFiles(modVal, force=force)
        arts.parseArtistModValExtraFiles(modVal)

from artist import artist
from artists import artists
arts = artists(disc)

#parseArtists(arts, True)
#arts.parseArtistFiles(force=False)
#if False:
#art.searchDiscogForArtist("$uicideboy$")
arts.buildMetadata(force=False, doAlbums=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Aug 22, 2019 21:03:59 for Building Artist Metadata DB
Loading data from /Volumes/Music/Discog/db/ArtistNameToID.p
  --> This file is 8.7MB.
Loading /Volumes/Music/Discog/db/ArtistNameToID.p
Loading data from /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
  --> This file is 20.0MB.
Loading /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 8.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistRefToID.p
  --> This file is 10.3MB.
Loading /Volumes/Music/Discog/db/ArtistRefToID.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 10.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistRefToName.p
  --> This file is 12.1MB.
Loading /Volumes/Music/Discog/db/ArtistRefToName.p


# Albums

In [3]:
def parseAlbums(modVal, force=False):
    albs.parseAlbumModValFiles(modVal=modVal, force=force)
    
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)


def f_amp(modVals=100):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, range(100))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
albs.parseAlbumModValFiles(modVal='NAN', force=False)
f_amp()
elapsed(start2, cmt2)
albs.buildMetadata(force=False)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Sep 07, 2019 10:02:14 for Parsing All Albums
Current Time is Sat Sep 07, 2019 10:02:14 for Parallel Album Parsing
Current Time is Sat Sep 07, 2019 10:02:14 for Parsing Album Files For ModVal NAN
  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
  Found 1 artist directories
    Found 75538 albums (0    ) in 0/1	/Volumes/Music/Discog/albums/NAN/NAN
Saving 634 new album IDs to /Volumes/Music/Discog/albums-db/NAN-DB.p
  --> This file is 66.4MB.
Current Time is Sat Sep 07, 2019 10:09:16 for Done with Parsing Album Files For ModVal NAN
Process [{0}] took 7.0 minutes.
Current Time is Sat Sep 07, 2019 10:09:18 for Parsing Album Files For ModVal 18
Current Time is Sat Sep 07, 2019 10:09:18 for Parsing Album Files For ModVal 9
Current Time is Sat Sep 07, 2019 10:09:18 for Parsing Album Files For ModVal 0
  Looking for artist albums in /Volumes/Music/Discog/albums/18
  Looking

    Found 1   albums (57   ) in 400/2855	/Volumes/Music/Discog/albums/1/1618201
    Found 1   albums (61   ) in 500/2855	/Volumes/Music/Discog/albums/1/1766501
    Found 0   albums (64   ) in 600/2855	/Volumes/Music/Discog/albums/1/1937501
    Found 7   albums (64   ) in 700/2855	/Volumes/Music/Discog/albums/1/209001
    Found 2   albums (78   ) in 800/2855	/Volumes/Music/Discog/albums/1/2241401
    Found 1   albums (78   ) in 900/2855	/Volumes/Music/Discog/albums/1/2405601
    Found 5   albums (238  ) in 600/1649	/Volumes/Music/Discog/albums/10/258410
    Found 3   albums (89   ) in 1000/2855	/Volumes/Music/Discog/albums/1/2583501
    Found 0   albums (241  ) in 700/1649	/Volumes/Music/Discog/albums/10/2929510
  Found 1603 artist directories
    Found 0   albums (0    ) in 0/1603	/Volumes/Music/Discog/albums/19/1000419
    Found 1   albums (94   ) in 1100/2855	/Volumes/Music/Discog/albums/1/2736701
    Found 3   albums (244  ) in 800/1649	/Volumes/Music/Discog/albums/10/3248510
    Fo

    Found 2   albums (250  ) in 1100/1599	/Volumes/Music/Discog/albums/20/485320
    Found 1   albums (250  ) in 1200/1599	/Volumes/Music/Discog/albums/20/5282720
    Found 4   albums (440  ) in 1300/2834	/Volumes/Music/Discog/albums/2/313402
    Found 7   albums (453  ) in 1400/2834	/Volumes/Music/Discog/albums/2/33202
    Found 0   albums (271  ) in 1300/1599	/Volumes/Music/Discog/albums/20/590520
    Found 2   albums (461  ) in 1500/2834	/Volumes/Music/Discog/albums/2/3505902
    Found 4   albums (273  ) in 1400/1599	/Volumes/Music/Discog/albums/20/691520
    Found 1   albums (465  ) in 1600/2834	/Volumes/Music/Discog/albums/2/3692002
    Found 0   albums (469  ) in 1700/2834	/Volumes/Music/Discog/albums/2/3896502
    Found 1   albums (473  ) in 1800/2834	/Volumes/Music/Discog/albums/2/4141502
    Found 0   albums (474  ) in 1900/2834	/Volumes/Music/Discog/albums/2/4445002
    Found 0   albums (878  ) in 700/1623	/Volumes/Music/Discog/albums/11/3092211
    Found 1   albums (296  ) i

    Found 2   albums (1089 ) in 800/2839	/Volumes/Music/Discog/albums/3/2155803
    Found 1   albums (1096 ) in 900/2839	/Volumes/Music/Discog/albums/3/2336703
    Found 7   albums (1098 ) in 1000/2839	/Volumes/Music/Discog/albums/3/251803
    Found 1   albums (1104 ) in 1100/2839	/Volumes/Music/Discog/albums/3/267703
    Found 1   albums (1104 ) in 1200/2839	/Volumes/Music/Discog/albums/3/2864203
    Found 30  albums (1105 ) in 1300/2839	/Volumes/Music/Discog/albums/3/306503
    Found 3   albums (1113 ) in 1400/2839	/Volumes/Music/Discog/albums/3/3228203
    Found 0   albums (259  ) in 500/1713	/Volumes/Music/Discog/albums/12/2260512
    Found 2   albums (1114 ) in 1500/2839	/Volumes/Music/Discog/albums/3/3449203
    Found 1   albums (1121 ) in 1600/2839	/Volumes/Music/Discog/albums/3/3616203
    Found 2   albums (1123 ) in 1700/2839	/Volumes/Music/Discog/albums/3/384103
    Found 1   albums (1125 ) in 1800/2839	/Volumes/Music/Discog/albums/3/4129603
    Found 1   albums (1128 ) in 19

    Found 1   albums (69   ) in 400/1670	/Volumes/Music/Discog/albums/13/1972113
    Found 1   albums (74   ) in 500/1670	/Volumes/Music/Discog/albums/13/2286013
    Found 24  albums (86   ) in 600/1670	/Volumes/Music/Discog/albums/13/264513
    Found 0   albums (98   ) in 700/1670	/Volumes/Music/Discog/albums/13/3060013
    Found 1   albums (102  ) in 800/1670	/Volumes/Music/Discog/albums/13/3356013
    Found 3   albums (102  ) in 900/1670	/Volumes/Music/Discog/albums/13/372013
  --> This file is 6.9MB.
Current Time is Sat Sep 07, 2019 10:19:45 for Done with Parsing Album Files For ModVal 23
Process [{0}] took 1.6 minutes.
Current Time is Sat Sep 07, 2019 10:19:45 for Parsing Album Files For ModVal 24
  Looking for artist albums in /Volumes/Music/Discog/albums/24
  --> This file is 4.5MB.
Current Time is Sat Sep 07, 2019 10:19:46 for Done with Parsing Album Files For ModVal 4
Process [{0}] took 1.2 minutes.
Current Time is Sat Sep 07, 2019 10:19:46 for Parsing Album Files For ModVal 5

...

    Found 1   albums (1    ) in 100/2416	/Volumes/Music/Discog/albums/93/1140593
    Found 1   albums (5    ) in 200/2416	/Volumes/Music/Discog/albums/93/1315693
    Found 0   albums (7    ) in 300/2416	/Volumes/Music/Discog/albums/93/1473093
    Found 1   albums (13   ) in 400/2416	/Volumes/Music/Discog/albums/93/1642693
    Found 2   albums (16   ) in 500/2416	/Volumes/Music/Discog/albums/93/1791793
    Found 1   albums (21   ) in 600/2416	/Volumes/Music/Discog/albums/93/1968593
  --> This file is 6.5MB.
Current Time is Sat Sep 07, 2019 11:23:34 for Done with Parsing Album Files For ModVal 85
Process [{0}] took 2.7 minutes.
Current Time is Sat Sep 07, 2019 11:23:34 for Parsing Album Files For ModVal 86
  Looking for artist albums in /Volumes/Music/Discog/albums/86
    Found 1   albums (35   ) in 700/2416	/Volumes/Music/Discog/albums/93/2154093
    Found 0   albums (997  ) in 1600/1637	/Volumes/Music/Discog/albums/78/933878
    Found 1   albums (37   ) in 800/2416	/Volumes/Music/Disco

Current Time is Sat Sep 07, 2019 11:27:29 for Parsing Album Files For ModVal 88
  Looking for artist albums in /Volumes/Music/Discog/albums/88
  --> This file is 7.3MB.
Current Time is Sat Sep 07, 2019 11:27:37 for Done with Parsing Album Files For ModVal 93
Process [{0}] took 4.5 minutes.
Current Time is Sat Sep 07, 2019 11:27:37 for Parsing Album Files For ModVal 94
  Looking for artist albums in /Volumes/Music/Discog/albums/94
  Found 1613 artist directories
    Found 38  albums (0    ) in 0/1613	/Volumes/Music/Discog/albums/80/1000280
    Found 2   albums (20   ) in 100/1613	/Volumes/Music/Discog/albums/80/1246280
  Found 1600 artist directories
    Found 2   albums (0    ) in 0/1600	/Volumes/Music/Discog/albums/88/1001988
    Found 1   albums (11   ) in 100/1600	/Volumes/Music/Discog/albums/88/1246788
    Found 14  albums (15   ) in 200/1600	/Volumes/Music/Discog/albums/88/1540088
    Found 3   albums (18   ) in 300/1600	/Volumes/Music/Discog/albums/88/1870888
    Found 3   albums

    Found 1   albums (1130 ) in 2300/2711	/Volumes/Music/Discog/albums/94/61494
    Found 0   albums (66   ) in 1500/2754	/Volumes/Music/Discog/albums/99/3607099
    Found 1   albums (271  ) in 1300/1672	/Volumes/Music/Discog/albums/89/5463789
    Found 1   albums (70   ) in 1600/2754	/Volumes/Music/Discog/albums/99/386999
    Found 4   albums (276  ) in 1400/1672	/Volumes/Music/Discog/albums/89/639889
    Found 7   albums (73   ) in 1700/2754	/Volumes/Music/Discog/albums/99/4159299
    Found 2   albums (76   ) in 1800/2754	/Volumes/Music/Discog/albums/99/4441499
    Found 0   albums (285  ) in 1500/1672	/Volumes/Music/Discog/albums/89/727089
    Found 1   albums (78   ) in 1900/2754	/Volumes/Music/Discog/albums/99/4751299
    Found 2   albums (80   ) in 2000/2754	/Volumes/Music/Discog/albums/99/5009599
    Found 1   albums (80   ) in 2100/2754	/Volumes/Music/Discog/albums/99/5278199
    Found 1   albums (81   ) in 2200/2754	/Volumes/Music/Discog/albums/99/5580799
    Found 2   albums 

  Looking for artist albums in /Volumes/Music/Discog/albums/97
  Found 2686 artist directories
    Found 1   albums (0    ) in 0/2686	/Volumes/Music/Discog/albums/97/1001097
    Found 5   albums (9    ) in 100/2686	/Volumes/Music/Discog/albums/97/11297
    Found 3   albums (15   ) in 200/2686	/Volumes/Music/Discog/albums/97/1293997
    Found 1   albums (15   ) in 300/2686	/Volumes/Music/Discog/albums/97/1426397
    Found 2   albums (19   ) in 400/2686	/Volumes/Music/Discog/albums/97/1580197
    Found 3   albums (24   ) in 500/2686	/Volumes/Music/Discog/albums/97/173497
    Found 1   albums (38   ) in 600/2686	/Volumes/Music/Discog/albums/97/192497
    Found 1   albums (45   ) in 700/2686	/Volumes/Music/Discog/albums/97/2093697
    Found 11  albums (53   ) in 800/2686	/Volumes/Music/Discog/albums/97/2273397
    Found 1   albums (62   ) in 900/2686	/Volumes/Music/Discog/albums/97/2470197
    Found 1   albums (66   ) in 1000/2686	/Volumes/Music/Discog/albums/97/2635497
    Found 1   album

  --> This file is 22.7MB.
Saved data to /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 22.7MB.
Saving 715041 entries to /Volumes/Music/Discog/db/AlbumIDToName.p
Saving data to /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 16.3MB.
Saved data to /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 16.3MB.
Saving 721053 entries to /Volumes/Music/Discog/db/AlbumRefToName.p
Saving data to /Volumes/Music/Discog/db/AlbumRefToName.p
  --> This file is 29.0MB.
Saved data to /Volumes/Music/Discog/db/AlbumRefToName.p
  --> This file is 29.0MB.
Saving 715041 known artists to /Volumes/Music/Discog/db/AlbumIDToArtistID.p
Saving data to /Volumes/Music/Discog/db/AlbumIDToArtistID.p
  --> This file is 11.2MB.
Saved data to /Volumes/Music/Discog/db/AlbumIDToArtistID.p
  --> This file is 11.2MB.
Saving 177000 known artists to /Volumes/Music/Discog/db/AlbumArtistMetaData.p
Saving data to /Volumes/Music/Discog/db/AlbumArtistMetaData.p
  --> This file is 22.5MB.
Saved data 

In [32]:
db = disc.getAlbumArtistMetaData()

Loading data from /Volumes/Music/Discog/db/AlbumArtistMetaData.p
  --> This file is 13.3MB.
Loading /Volumes/Music/Discog/db/AlbumArtistMetaData.p


In [37]:
db['81013']['Genre'].most_common(1)[0][0]

'Rock'

In [16]:
for k,v in db.items():
    print(k,v)
    break

Dave Clarke 1000


In [ ]:
f1="/Volumes/Music/Discog/albums/6/102506/482739.p"
#f2="/Volumes/Music/Discog/albums/6/102506/7303693.p"
#albData1 = alb.getData(f1)
f2="/Volumes/Music/Discog/albums/11/567511/3386791.p"
albData2 = alb.getData(f2)

In [ ]:
albData1.album

In [ ]:
for k,v in albData2.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
for k,v in albData1.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
from album import album
from fileUtils import getsize
alb = album()
for ifile in files:
    print(getsize(ifile))
    #alb.getData(ifile)

In [ ]:
for modVal in range(58,59):
    arts.parseArtistModValFiles(modVal=modVal, force=False)

dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
from time import sleep
refs = [dbCol.get('716158')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if True:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_mp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map(f, chunks)
 
    return sum(result)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
 
    while not result.ready():
        print("Running...")
        time.sleep(0.5)
 
    return sum(result.get())

In [ ]:
L = [{0: True}, {1: True}, {2: True}, {3: True}, {4: True}]

In [ ]:
{k: v for d in L for k, v in d.items()}

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()
artistInfo = artist()
artistDBDir = disc.getArtistsDBDir()   

In [ ]:
len(files)

In [ ]:
from fileUtils import getBaseFilename
import time
from multiprocessing import Pool

def f(finput):
    a_list = finput["Data"]
    dbdata = finput["DB"]
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out

def fDB(modVal):
    saveIt = arts.parseArtistModValFiles(modVal, force=True)
    
    if False:
        from searchUtils import findExt
        files = findExt(setDir(artistDir, str(modVal)), ext="*.p")
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        dbdata = getFile(dbname, version=3)

        out = []
        saveIt = 0
        for ifile in files:
            discID = getBaseFilename(ifile)
            if dbdata.get(discID) is None:
                info   = artistInfo.getData(ifile)
                dbdata[discID] = info
                saveIt += 1

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
    return saveIt

 
def f_ampFiles(a_list, dbdata):
    nProcs = 1
    chunks = [a_list[i::nProcs] for i in range(nProcs)]

    for c in chunks:
        print(len(c))
    
    pool = Pool(processes=nProcs)
    
    
 
    finputs = [{"Data": chunk, "DB": dbdata} for chunk in chunks]
    result = pool.map_async(f, finputs)
 
    start, cmt = clock("Processing {0} files using {1} processes".format(len(files), nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()



 
def f_amp(modVals):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(fDB, range(3,modVals))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
from timeUtils import clock, elapsed
retval = f_amp(100)
#flat_list = [item for sublist in retval for item in sublist]
#print(len(flat_list))

In [ ]:
dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
dbdata = getFile(dbname, version=3)

In [ ]:
modVals = 1
nProcs = 1
start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
print("Runing .",end="")
retval = fDB(modVals)
print("")
elapsed(start, cmt)


In [ ]:
retval

In [ ]:
vals="""1,708,466 US
1,708,466 UK
1,154,284 Germany
637,261 France
493,079 Japan
490,698 Italy
434,681 Canada
433,543 Europe
416,965 Netherlands
339,738 Spain
320,714 Unknown
259,587 Russia
259,579 Australia
189,431 Sweden
164,278 Belgium
149,548 Brazil
131,106 Greece
124,488 Poland
121,249 Jamaica
105,887 Finland
86,538 Switzerland
85,145 USSR
80,203 Mexico
79,972 Denmark
77,131 Portugal
75,979 Norway
75,533 UK & Europe
67,292 Austria
63,672 Argentina"""

In [ ]:
countries = [" ".join(x.split()[1:]) for x in vals.split("\n")]
saveFile(idata=countries, ifile="countries.p")

In [ ]:
countries